In [1]:
### Read the raw file to preprocess
import pandas as pd
final_raw=pd.read_csv('pubchem_new_0_10000_new.csv')

In [2]:
print(len(final_raw))

1


In [3]:
sol_new=final_raw.Solubility
import re
sol1=[]
### Using regular expression module cleaning the data and taking out the data with with unit "mg/mL" in the pubchem ...
for s in sol_new:
   sol=re.findall('(?<![\-\/])(\d+(?:\.?\d+)).*( ug/mL|mg/mL|(ug/mL)|mg/L|ml)', s)
   sol1.append(sol)
print(len(sol1))


1


In [4]:
### Making sure that no null values present in the dataframe 
df3=final_raw.SMILES
df3.reset_index(drop=True, inplace=True)
df2 = pd.DataFrame(sol1, columns = ['Solubility'])
df2 = pd.DataFrame(sol1)
df_new = pd.concat([df3,df2], axis=1,ignore_index=True)
df_new.columns =['SMILES', 'SOLUBILITY']
df_sol = df_new.loc[df_new['SOLUBILITY'].notnull(), ['SMILES','SOLUBILITY']]
print(len(df_sol))
df_sol1=df_sol.reset_index()

1


In [5]:
df2=pd.DataFrame()
for sol in df_sol.SOLUBILITY:

  df=pd.DataFrame(sol)
  df=df.T
  df1=df.iloc[:,0:2]
  df2=pd.concat([df2,df1],axis=0,ignore_index=True)
df_final=pd.concat([df_sol1,df2],axis=1)
df_final.rename(columns={ df_final.columns[3]: "sol_new1" }, inplace = True)
df_final.rename(columns={ df_final.columns[4]: "Unit" }, inplace = True)
df_final.sol_new1 = pd.to_numeric(df_final.sol_new1, errors="coerce")
df_final['sol_final']=''
#### based on the unit converting the solubilities values 
for i in range (len(df_final)):

  if df_final['Unit'][i]=='ug/mL':
    df_final['sol_final'][i]=(df_final['sol_new1'][i]/1000).round(5)
  elif df_final['Unit'][i]=='mg/mL':
    df_final['sol_final'][i]=df_final['sol_new1'][i]
  elif df_final['Unit'][i]=='mg/L':
    df_final['sol_final'][i]=(df_final['sol_new1'][i]/1000).round(5)
  else:
    df_final['sol_final'][i]=(df_final['sol_new1'][i]/1000).round(5)
df_final

/var/folders/k_/8s01c4dj0gnfx2n284nvm1t80000gn/T/ipykernel_45830/315374829.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['sol_final'][i]=(df_final['sol_new1'][i]/1000).round(5)


,index,SMILES,SOLUBILITY,sol_new1,Unit,sol_final
0,0,CCCCOC1=CC=C(C=C1)CC(=O)NO,"(33.5, ug/mL, ug/mL)",33.5,ug/mL,0.0335


In [7]:
### saving the clean data to the disk 
df_final.to_csv('pubchem_clean_0k_10000k.csv')